# VibraCore report

______________________________________________________________________

**Authors: Robin Wimmers  (r.wimmers@cemsbv.io)**

Notebook with stepwise explanation of reading and processing CPT data and writing the results to a standard template. For more information about VibraCore, take a look at the documentation!

The structure of the tutorial is as follows:

- [Project definition](#Input-definition)
- [Download CPT's in project](#Download-CPTs-in-project)
- [Classify CPTs](#Classify-CPTs)
- [Call VibraCore-API](#Call-VibraCore-API)
- [View Results](#View-Results)
- [Report](#Report)


Now we can import the required packages:

In [ ]:
import os
from typing import Dict
import io
import logging

import pygef
from nuclei.client import NucleiClient
from pyvibracore import api
from pyvibracore.input.impact_force_properties import (
    VibrationSource,
    create_multi_cpt_impact_force_payload,
    create_multi_cpt_impact_force_report_payload,
)
from pyvibracore.results.impact_force_result import MultiCalculationData

import contextily as ctx
import pandas as pd
from tqdm import tqdm

pd.set_option("display.max_columns", None)
logging.getLogger().setLevel(logging.INFO)

### Start a Nuclei client session

In the next cell we will create a nuclei-client with a session that takes care of the
authentication and communication with the Nuclei server.

You will need to provide your user token, which can be obtained by login in to the [nuclei website](https://nuclei.cemsbv.io/) with your personal credentials and going to the "API Access Tokens" section.

In [ ]:
# os.environ["NUCLEI_TOKEN"] = "<YOUR TOKEN>"

client = NucleiClient()

# Input definition

<div style="background: #f2ed4c;
            width: 100%;
            color: black;
            text-align: center;">
<b>USER INPUT REQUIRED BELOW:<b>
</div>

In [ ]:
# General input data
project_id = 21305  # also used to select cpt
project_remark = "Voorbeeld Notebook"  # Optional additional information
author = "N. Uclei"
project_name = "Impact force calculation"

In [ ]:
# Specify CPT selection from project

# ** cpt_selection
# Specify a list of CPTs names (i.e. BRO ID).
cpt_selection = [
    "CPT000000200908",
    "CPT000000200909",
    "CPT000000112268",
    "CPT000000112278",
    "CPT000000200904",
    "CPT000000200905",
    "CPT000000112265",
    "CPT000000200906",
    "CPT000000200907",
]


# ** classify_metode:
# Metode used to classify CPT data.
# Accepted values: ["beenJefferies", "machineLearning", "nen", "table", "robertson", "ntype"]
classify_metode = "ntype"

# ** groundwater_level
# The groundwater-level in the project [m w.r.t offset].
# When None, the water-level of the CPTs is used.
groundwater_level_offset = -0.5

In [ ]:
# Sheet pile information (single sheetpile (1 sheet))
name = "AZ12-770"

# ** slot_resistance_specific
# kPa push = 5 / vibrate = 10 for new sheet piles
slot_resistance_specific = 5

# ** amount_of_sheet_piles
# push = 1 / vibrate = 2
amount_of_sheet_piles = 2

In [ ]:
# Installation information

# ** drive_strategy
# Accepted values: [push, vibrate]
drive_strategy = "vibrate"

# ** installation_level_offset
# installation level [m w.r.t offset]
installation_level_offset = -20

In [ ]:
# ** friction_strategy
# Defines the the strategy on how to computed the sleeve friction.
# Accepted values: [SlipFrictionStrategy, CPTFrictionStrategy]
friction_strategy = "SlipFrictionStrategy"

<div style="background: #f2ed4c;
            width: 100%;
            color: black;
            text-align: center;">
<b>END USER INPUT<b>
</div>

#### Download CPTs in project

In [ ]:
# Get CPTs
# loop over the cpt id's and fetch file from BRO
cptdata_objects = []
for file_metadata in tqdm(cpt_selection, desc="Download CPT's from BRO"):
    # download CPT from BRO
    response = client.session.get(
        url=f"https://publiek.broservices.nl/sr/cpt/v1/objects/{file_metadata}"
    )
    if not response.ok:
        print(
            f"RuntimeError: {file_metadata} could not be donwloaded from de BRO server. \n Statuse code: {response.status_code}"
        )
        continue

    cpt = pygef.read_cpt(io.BytesIO(response.content))
    object.__setattr__(cpt, "alias", file_metadata)
    object.__setattr__(cpt, "data", cpt.data.drop_nulls())
    cptdata_objects.append(cpt)

#### Classify CPTs

In [ ]:
classify_tables: Dict[str, dict] = {}

for i, cpt in tqdm(enumerate(cptdata_objects), desc="Classify CPT's from BRO"):
    # classify CPT with CPTCore
    payload = {
        "aggregateLayersPenalty": 5,
        "minimumSegmentLength": 5,
        "data": {
            "coneResistance": cpt.data.get_column("coneResistance")
            .clip(0, 50)
            .to_list(),
            "correctedPenetrationLength": cpt.data.get_column("depth").to_list(),
            "localFriction": cpt.data.get_column("localFriction").clip(0, 50).to_list(),
        },
        "verticalPositionOffset": cpt.delivered_vertical_position_offset,
        "x": cpt.delivered_location.x,
        "y": cpt.delivered_location.y,
    }
    if "porePressureU2" in cpt.data.columns:
        payload["data"]["porePressureU2"] = (
            cpt.data.get_column("porePressureU2").clip(0, 50).to_list(),
        )[0]

    response = client.session.post(
        f"https://crux-nuclei.com/api/cptcore/v1/classify/{classify_metode}",
        json=payload,
    )
    if not response.ok:
        cptdata_objects.pop(i)
        print(
            f"RuntimeError: {file_metadata} could not be classified. \n Statuse code: {response.status_code}"
        )
        continue
    classify_tables[cpt.alias] = response.json()

#### Call VibraCore-API

In [ ]:
# get pile properties
vibration_source = VibrationSource.from_sheet_pile_name(
    name, amount_of_sheet_piles, slot_resistance_specific
)

In [ ]:
# Get results
multi_cpt_payload = create_multi_cpt_impact_force_payload(
    cptdata_objects=cptdata_objects,
    classify_tables=classify_tables,
    vibration_source=vibration_source,
    friction_strategy=friction_strategy,
    drive_strategy=drive_strategy,
    installation_level_offset=installation_level_offset,
    groundwater_level_offset=groundwater_level_offset,
)

api_response = api.get_impact_force_calculation(
    client=client, payload=multi_cpt_payload
)

results = MultiCalculationData.from_api_response(
    response_dict=api_response,
)

## View Results

In [ ]:
fig = results.plot(figsize=[20, 15])

# add basemap
ctx.add_basemap(fig.axes[0], crs="EPSG:28992", source=ctx.providers.Esri.WorldTopoMap)

## Report

In [ ]:
# Create report
report_payload = create_multi_cpt_impact_force_report_payload(
    multi_cpt_payload,
    project_name=project_name,
    project_id=str(project_id),
    author=author,
)

report = api.get_impact_force_report(client=client, payload=report_payload)

with open(f"{project_name} report.pdf", "wb") as f:
    f.write(report)